In [1]:
import tracktable
import pandas as pd
import os.path




from tracktable.core import geomath
from tracktable.domain.terrestrial import TrajectoryPointReader
from tracktable.applications.assemble_trajectories import AssembleTrajectoryFromPoints
from tracktable.render.render_trajectories import render_trajectories

from datetime import datetime, timedelta


In [2]:
print(tracktable.__version__)
print("All libraries imported!")

data_filename = ("/anvil/projects/tdm/corporate/sandia-trajectory/previous_files/flight/data/raw_data/oneDay.tsv")
inFile = open(data_filename, 'r')
reader = TrajectoryPointReader()
reader.input = inFile
reader.comment_character = "#"	#What character is used for comments
reader.field_delimiter = "\t"	#What character "breaks" each data value ex: Comma-Separated Values


1.7.0
All libraries imported!


In [3]:
#Columns start at 0, ex: 0 is column A, 2 is column C
reader.object_id_column = 0 	#What column holds the object ID
reader.timestamp_column = 1 	#What column holds the timestamp
reader.coordinates[1] = 3		#What column holds LAT data
reader.coordinates[0] = 2		#What column holds LONG data
reader.set_real_field_column('speed', 4) #Extra data (heading)
reader.set_real_field_column('heading', 5) #Extra data (heading)
reader.set_real_field_column('altitude', 6) #Extra data (altitude)
reader.set_string_field_column('ac_type', 7)
reader.set_string_field_column('departure', 8) #Dearture Airport
reader.set_string_field_column("destination", 9) #Destination Airport

In [4]:
#Combine datapoints together using the object_id
builder = AssembleTrajectoryFromPoints()
builder.input = reader
builder.minimum_length = 10
builder.separation_time = timedelta(minutes=30)
traj = list(builder.trajectories())

INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:New trajectories will be declared after a separation of None distance units between two points or a time lapse of at least 0:30:00 (hours, minutes, seconds).
INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Trajectories with fewer than 10 points will be discarded.


[2024-02-16 13:36:20.890765] [0x00007f5978ef5080] [info]    Done reading points. Generated 4563365 points correctly and discarded 1 due to parse errors.



INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Done assembling trajectories. 40868 trajectories produced and 1922 discarded for having fewer than 10 points.


In [5]:
flightOfInterest = traj[3000]



flightLength = tracktable.core.geomath.length(flightOfInterest)

convexHullRatio = tracktable.core.geomath.convex_hull_aspect_ratio(flightOfInterest)

In [6]:
departures = []
destinations = []
for flight in traj:
    departures.append(flight[0].properties['departure'])
    destinations.append(flight[0].properties['destination'])
    # for point in flight:
    #     departures.append(point.properties["departure"])
    #     destinations.append(point.properties["destination"])
depDF = pd.DataFrame(departures)
destDF = pd.DataFrame(destinations)

print(depDF.head())


        0
0   NK310
1  5T1524
2   B6100
3  Y87465
4   B6261


In [7]:
tracktable.render.render_trajectories.render_trajectories(flightOfInterest, backend='', simplify_traj=False, simplify_tol=0.0001)





In [7]:
listT = [flightOfInterest, traj[2000]]
tracktable.render.render_trajectories.render_trajectories(traj[2000], backend='', simplify_traj=False, simplify_tol=0.0001, gradient_hue=0.5)

#Each number for graidient hue means different color; however, not all numbers work. 

In [8]:
tracktable.render.render_trajectories.render_trajectories(listT, backend='', simplify_traj=False, simplify_tol=0.0001, gradient_hue = [0.5,0.2])


#When wanting trajectories to say different things, we can put a list of items that are equivalent to the 



In [9]:
def visualize(listOfTrajectories, ranked):
    if ranked is False:
        return tracktable.render.render_trajectories.render_trajectories(listOfTrajectories, backend='', simplify_traj=False, simplify_tol=0.0001)
    else:
        
        possible_colors = [0.1, 0.5, 0.5, 1.0, 1.0]
        colors = []
        if len(listOfTrajectories) > 5:
            
            colors = possible_colors
            for i in range(5, len(listOfTrajectories) - 1):
                colors.append(1.2)
        
        if len(listOfTrajectories) == 5:
            colors = possible_colors
        else:
            for i in range(len(listOfTrajectories)):
                colors.append(possible_colors[i])
                
        return tracktable.render.render_trajectories.render_trajectories(listOfTrajectories, backend='', simplify_traj=False, simplify_tol=0.0001, gradient_hue = colors)
        
                

        
    

In [13]:
test_list = [traj[3000], traj[2001], traj[708], traj[405], traj[1], traj[98]]
visualize(test_list, False)


In [14]:

tracktable.render.render_trajectories.render_trajectories(listT, backend='', simplify_traj=False, simplify_tol=0.0001, save)
        
    

In [ ]:
#WE CANNOT CALL RENDER TRAJECTORIES INSIDE OF A FUNCTION, SO WE CAN EITHER CALL IT OUTSIDE, OR GENERATE A STATIC IMAGE. 